# Projet 1, TP4 - Apprentissage par lots

On continue à suivre [le tutoriel PyTorch](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#full-implementation). On remarque notamment que la boucle d'apprentissage ressemble à la notre :

```py
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
```

Cependant lorsqu'on regarde l'implémentation de `train_loop`, il contient une autre boucle,

```py
for batch, (X, y) in enumerate(dataloader):
```

C'est quoi un `dataloader`? Et pourquoi est-ce qu'il y a une boucle dans la boucle ?

Précédemment, on a donné fait une descente de gradient en prenant toutes les données en entrée du réseau. 
En pratique, pour beaucoup d'applications (par exemple des images), cette quantité d'information est bien trop importante pour la mémoire de notre ordinateur, et il faut traiter les données **par lots** (par *batches*).
Ainsi, dans le tutoriel, on trouve une ligne `train_dataloader = DataLoader(training_data, batch_size=64)`, qui initialise un itérable qui sert à obtenir un lot après l'autre, en l'occurrence avec des sous-échantillons de taille 64.
Un passage complet dans cette boucle interne s'appelle une **époque** (ou *epoch*).

Dans ce TP, on va étudier ce que ce comportement change 

*Remarque :* Le tutoriel contient aussi un `test_loop` et un `test_dataloader`. Ici, on n'a pas besoin de distinguer les jeux de données d'«entraînement» et de «validation», parce que notre **représentation** est très bien choisie. Il n'y a pas de risque d'*overfitting*.


In [2]:
# Quelques imports utiles

import numpy as np
from numpy.polynomial.polynomial import Polynomial

import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

import projet1 as p1

In [3]:
x_data, y_data = p1.get_dataset()
dataset = TensorDataset(x_data, y_data)

slope_plot = np.linspace(-10, 10, 40)
bias_plot = np.linspace(-10, 10, 30)
loss_plot = p1.build_loss(slope_plot, bias_plot, x_data, y_data)

data_trace = p1.data_scatter_trace(x_data, y_data)
loss_trace = p1.init_contour_trace(slope_plot, bias_plot, loss_plot)

## Partie 1 - Introduction aux batches

### 1.1. Apprentissage sur une époque

**TODO:**  
Modifier cette fonction pour sauvegarder les valeurs des paramètres (pente et biais) du modèle après chaque pas.

In [4]:
def train_epoch(dataloader, model, loss_fun, optimizer):
    for x_batch, y_batch in dataloader:
        optimizer.zero_grad()

        y_pred = model(x_batch)
        batch_loss = loss_fun(y_pred, y_batch)

        batch_loss.backward()
        optimizer.step()

        #TODO sauvegarde des valeurs des paramètres

    return None

### 1.2. La boucle d'apprentissage

**TODO:**  
Construire un *dataloader* avec des lots de la même taille que le nombre de données et appliquer une boucle d'apprentissage sur 20 époques. Vérifier que les résultats correspondent à la boucle d'apprentissage sans *dataloader* du TP précédent.

In [5]:
def train_linreg(
    n_epochs=100, batch_size=0, optim_algo=torch.optim.SGD, model=None, **optim_params
):
    if model is None:
        model = p1.LinearRegression()

    dataset = TensorDataset(*p1.get_dataset())
    if batch_size == 0:
        batch_size = len(dataset)

    #TODO définir le dataloader

    loss_fun = nn.MSELoss()
    optimizer = optim_algo(model.parameters(), lr=learning_rate)

    w_history, b_history = [], []
    for _ in range(n_epochs):
        w_epoch, b_epoch = train_epoch(dataloader, model, loss_fun, optimizer)
        #TODO enregistrer l'historique des paramètres

    return model, w_history, b_history

### 1.3. Des lots de taille maximale

**TODO:**  
Vérifier qu'avec `batch_size=0`, on retrouve bien les résultats du TP précédent.

In [ ]:
_, all_w, all_b = train_linreg(n_epochs=15)
p1.training_evolution(all_w, all_b, loss_trace=loss_trace, data_trace=data_trace)

### 1.4. L'apprentissage par lots

Faire tourner des apprentissages avec les paramètres suivants :

- `n_epochs=7, batch_size=25, learning_rate=5e-2`
- `n_epochs=7, batch_size=5, learning_rate=1e-2`
- `n_epochs=15, batch_size=1, learning_rate=1e-3`

Que remarquez-vous ?

## Partie 2 - À chaque batch son minimum

### 2.1. Mettre en valeur quelques batches

**TODO:**  
Pour les 4 premiers lots avec une `batch_size` de 10, calculer le minimum associé avec `Polynomial.fit`, et les stocker. Utiliser ensuite la fonction d'affichage `training_evolution` pour afficher ces minima et les modèles associés.

### 2.2. Afficher la loss de plusieurs batches


**TODO:**  
Pour différents batches, afficher la loss dans l'espace des paramètres. Que remarquez-vous ?
Faire évoluer le `batch_size`, notamment prendre `batch_size=1` et `batch_size=5` pour comparer.

In [41]:
w = np.linspace(-10, 10, 25)
b = np.linspace(-10, 10, 15)


def plot_first_batches(dataset, rows=3, cols=3, batch_size=10):
    dataloader = DataLoader(dataset, batch_size=batch_size, generator=p1.torch_rng())
    assert len(dataloader) >= rows * cols

    fig = make_subplots(rows=rows, cols=cols)
    min_w, min_b, mean_loss = [], [], 0.0
    for idx, (x_batch, y_batch) in enumerate(dataloader):
        i, j = divmod(idx, cols)
        #TODO calculer la loss et l'afficher
    return fig

In [ ]:
plot_first_batches(dataset, batch_size=5)

## Partie 3 - Descente avec inertie

Pour s'extraires des minimas locaux, on peut ajouter un terme d'inertie à la descente de gradient. Cela permet d'explorer un peu plus l'espace des paramètres, en espérant trouver un minimum global, et de diminuer l'impact des différences entres les lots.

### 3.1. L'algorithme Adam

**TODO:**  
Reprendre l'algorithme de descente précédent, mais en utilisant l'optimiseur `torch.optim.Adam`, avec un *learning rate* de 1 et une *batch size* de 10. Comparer les résultats obtenus.

*Remarque :* On peut aussi préciser un argument `momentum` pour l'inertie dans `torch.optim.SGD`.

### 3.2. Sauvegarde du meilleur modèle

Le risque de ces méthodes est de s'extraire du minimum global. Pour éviter cela, on peut garder un œil sur la loss moyenne pendant une époque, et sauvegarder le modèle qui a la meilleure performance.

**TODO:**  
Reprendre l'algorithme précédent en implémentant cette sauvegarde avec `torch.save`.